# Query Data with AWS Data Wrangler
* https://github.com/awslabs/aws-data-wrangler
* https://aws-data-wrangler.readthedocs.io

In [ ]:
!pip install -q awswrangler==1.2.0

In [ ]:
import boto3
import sagemaker

# Get region 
session = boto3.session.Session()
region_name = session.region_name

# Get SageMaker session & default S3 bucket
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

In [ ]:
import awswrangler as wr

# Query Parquet from S3 with Push-Down Filters

In [ ]:
path = '{}/amazon-reviews-pds/parquet/'.format(bucket)
df_parquet_results = wr.s3.read_parquet(path,
                                     columns=['star_rating', 'product_category', 'review_body'],
                                     filters=[("product_category", "=", "Digital_Software")],
                                     dataset=True)
df_parquet_results.shape

In [ ]:
df_parquet_results.head(5)

# Query Parquet from S3 in Chunks

In [ ]:
path = '{}/amazon-reviews-pds/parquet/'.format(bucket)
chunk_iter = wr.s3.read_parquet(path,
                                columns=['star_rating', 'product_category', 'review_body'],
                                filters=[("product_category", "=", "Digital_Software")],
                                dataset=True,
                                chunked=True)

In [ ]:
print(next(chunk_iter))

# Query the Glue Catalog (ie. Hive Metastore)

In [ ]:
database_name = 'dsoaws'
table_name_tsv = 'amazon_reviews_tsv'

In [ ]:
for table in wr.catalog.get_tables(database="dsoaws"):
    print(table['Name'])

# Query from Athena

In [ ]:
%%time
df = wr.athena.read_sql_query(
    sql='SELECT * FROM {} LIMIT 5000'.format(table_name_tsv),
    database=database_name
)

In [ ]:
df.head(5)

# Query from Athena in Chunks
Retrieving in chunks can help reduce memory requirements

In [ ]:
%%time

chunk_iter = wr.athena.read_sql_query(
    sql='SELECT * FROM {} LIMIT 5000'.format(table_name_tsv),
    database='{}'.format(database_name),
    chunksize=64_000  # 64 KB Chunks
)

In [ ]:
print(next(chunk_iter))

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();